In [1]:
## Import needed modules ##

import pandas as pd
import os 

In [2]:
## Set the log header ##
## Below is Log entry format 2.0 ##
## You can find header in this document: https://docs.microsoft.com/en-us/rest/api/storageservices/storage-analytics-log-format ##

header=['<version-number>', '<request-start-time>', '<operation-type>',
       '<request-status>', '<http-status-code>', '<end-to-end-latency-in-ms>',
       '<server-latency-in-ms>', '<authentication-type>',
       '<requester-account-name>', '<owner-account-name>', '<service-type>',
       '<request-url>', '<requested-object-key>', '<request-id-header>',
       '<operation-count>', '<requester-ip-address>',
       '<request-version-header>', '<request-header-size>',
       '<request-packet-size>', '<response-header-size>',
       '<response-packet-size>', '<request-content-length>', '<request-md5>',
       '<server-md5>', '<etag-identifier>', '<last-modified-time>',
       '<conditions-used>', '<user-agent-header>', '<referrer-header>',
       '<client-request-id>', '<user-object-id>', '<tenant-id>',
       '<application-id>', '<audience>', '<issuer>', '<user-principal-name>',
       '<reserved-field>', '<authorization-detail>']

In [3]:
## Read all the subfolders ##

directory = "C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\"
folder_list = []

for root, subdirectories, files in os.walk(directory):
    for subdirectory in subdirectories:
        folder_list.append(os.path.join(root, subdirectory))
folder_list = folder_list[4:]
print(folder_list)

['C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0000', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0100', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0200', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0300', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0400', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0500', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0600\\0600', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0600\\0700', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0600\\0800', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0600\\0900', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0600\\1000', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0600\\1100', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\1200\\1200', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\1200\\1300', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\1200\\1400', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\1200\\1500', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\1200\\1600', 'C:\\Users\\z

In [6]:
## Read all the log files ##

log_list = []

for subfolder in folder_list:
    file_list = []
    for root, subdirectories, files in os.walk(subfolder):
        for file in files:
            string = os.path.join(root, file)
            if string[-4:] == ".log":
                file_list.append(os.path.join(root, file))
    log_list.append(file_list)
print(log_list)

[['C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0000\\000000.log', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0000\\000001.log', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0000\\000002.log', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0000\\000003.log', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0000\\000004.log', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0000\\000005.log', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0000\\000006.log', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0000\\000007.log', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0000\\000008.log', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0000\\000009.log'], ['C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0100\\000000.log', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0100\\000001.log', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0100\\000002.log', 'C:\\Users\\zoeylan\\AAA_Zoey_Working\\Logs\\0000\\0100\\000003.log', 'C:\\Users\\zoey

In [8]:
# pd.DataFrame(log_list)

In [9]:
## Create two empty dataframes to save final results ##
## Two examples here: one table for requester-ip-address and one for user-object-id ##

iptable = pd.DataFrame(columns= ['<requester-ip-address>', '<operation-type>'])
objectidtable = pd.DataFrame(columns=['<user-object-id>', '<operation-type>'])

print(iptable)
print(objectidtable)

Empty DataFrame
Columns: [<requester-ip-address>, <operation-type>]
Index: []
Empty DataFrame
Columns: [<user-object-id>, <operation-type>]
Index: []


In [13]:
## Read in all the logs and combine together in one table ##
## Do some filtering and group by work ##

i = 1 # Used to check the progress, should be 24 in total here

for hourlylog in log_list:
    
    log_df = pd.DataFrame(columns=header)
    print("Hour No.", i)
    
    for log in hourlylog:
        df = pd.read_csv(log, sep=';', low_memory=False, names = header)
        log_df = log_df.append(df, ignore_index=True)
        
        ## Filter out "AppendFile" operation only as an example
        write = log_df[log_df['<operation-type>'] == 'AppendFile']
        
        ## Count the total number of requests based on user-object-id & requester-ip-address
        objectid = pd.DataFrame(write.groupby(['<user-object-id>'])['<operation-type>'].count()).reset_index()
        ip = pd.DataFrame(write.groupby(['<requester-ip-address>'])['<operation-type>'].count()).reset_index()
        
        objectidtable = objectidtable.append(objectid, ignore_index=True)
        iptable = iptable.append(ip, ignore_index=True)
        
    i = i+1   

Hour No. 1
Hour No. 2
Hour No. 3
Hour No. 4
Hour No. 5
Hour No. 6
Hour No. 7
Hour No. 8
Hour No. 9
Hour No. 10
Hour No. 11
Hour No. 12
Hour No. 13
Hour No. 14
Hour No. 15
Hour No. 16
Hour No. 17
Hour No. 18
Hour No. 19
Hour No. 20
Hour No. 21
Hour No. 22
Hour No. 23
Hour No. 24


In [17]:
#iptable

In [19]:
## Remove the duplicates and sum up the count ##

iptable = pd.DataFrame(iptable.groupby(['<requester-ip-address>'])['<operation-type>'].sum()).reset_index()
objectidtable = pd.DataFrame(objectidtable.groupby(['<user-object-id>'])['<operation-type>'].sum()).reset_index()

In [20]:
objectidtable

,<user-object-id>,<operation-type>
0,106629de-c125-43a6-a27f-5fbdaccb5f82,458161
1,121bbaee-5726-4cf7-9eb2-c22102782414,800
2,1f900c3d-1fd6-46d7-9a15-309a48e397dc,802
3,2534029c-e85c-4adf-a17b-5027cf25b9c8,2233
4,3d76f635-29ca-4357-9c46-43669b35ca49,273229
5,4b2dd1f2-8c74-4a87-a8ed-735ce872d679,1790
6,5df091ca-31a3-4aab-9551-7d238db7b2d2,2910
7,702a5945-2416-4aff-b983-1b770bcb95fb,479
8,70ec36c1-d728-4940-ab57-8cca084b9f1b,240838
9,7a20c3b1-6eae-444e-a27d-12bc8e5a35ee,4578


In [ ]:
## Export as csv files ##

iptable.to_csv('iptable.csv')
objectidtable.to_csv('objectidtable.csv')

In [ ]:
objectidtable.to_csv('objectidtable.csv')

In [ ]:
file = open('C:\\Users\\zoeylan\\AAA_Zoey_Working\\Cx_Sample_Log\\000000Copy10rows.txt', 'r')
lines = file.read().splitlines()

for line in lines:
    if not line:
        continue
    newline = line.replace("&amp;", "&")
    newline2 = newline.replace('"', "'")
    #print(newline2)
    #columns = [col.strip() for col in newline2.split(';') if col]
    columns = newline2.split(";")
    list.append(columns)
    print(columns)
    #print (line)
    # do something
#file.close()

In [ ]:
for row in list:
    for column in row:
        if column == '':
           # print(type(column))
            column = 'nan'
            #print(column)
            
print(list[2:3])

In [ ]:
df = pd.DataFrame(list)
df = df.rename(columns=df.iloc[0]).drop(df.index[0])

In [ ]:
pd.DataFrame.from_records(list).head(10)

In [ ]:
pd.set_option('max_columns', None)
df.iloc[: 10, :]

In [ ]:
df = df.drop(df.index[df['<operation-count>'] != '0'])

In [ ]:
df.iloc[: 3, : 40]

In [ ]:
print(df.shape)

In [ ]:
print(df['<user-object-id>'].drop_duplicates())

In [ ]:
# filter out write operation - appendFile
#WriteOperation = df[df['<operation-type>'] == 'AppendFile']
wo = df[['<operation-type>', '<request-version-header>']]

WriteOperation = wo[wo['<operation-type>'] == 'AppendFile']
#WriteOperation.head(10)

In [ ]:
unique = WriteOperation.drop_duplicates()
unique

In [ ]:
print(unique.shape)

In [ ]:
txt = "2.0;2022-03-30T06:00:00.0000042Z;ReadFile;OAuthSuccess;200;16;16;bearer;phkprdeaspcommadls;phkprdeaspcommadls;blob;'https://phkprdeaspcommadls.dfs.core.windows.net/curatedzone/trafix_lite%2Fcsc%2Fcsc_traffic_count_trafixlite%2FBlock3_20171020.csv';'/phkprdeaspcommadls/curatedzone/trafix_lite/csc/csc_traffic_count_trafixlite/Block3_20171020.csv';03db8f00-e01f-003a-03fb-43e379000000;0;10.227.133.17;2018-11-09;1828;0;350;1098;0;;;'gzip';Monday, 15-Mar-21 06:14:17 GMT;;'Go/go1.16.8 (amd64-linux) go-autorest/v14.2.1 Azure-SDK-For-Go/v32.1.0 storagedatalake/2018-11-09';;'a7dbd65d-45ae-49a5-a667-2b9636fb3a35';'e3c4c6ba-fc3e-4a6b-aae8-401a1463a5e9';'33893f4e-3397-4cab-bb3c-9224bf7eafba';'67c60ae1-57cd-4296-be33-ce1f6f7b3636';'https://storage.azure.com/';'https://sts.windows.net/33893f4e-3397-4cab-bb3c-9224bf7eafba/';;;'[{&quot;action&quot;:&quot;Microsoft.Storage/storageAccounts/blobServices/containers/blobs/read&quot;, &quot;roleAssignmentId&quot;:&quot;&quot;, &quot;roleDefinitionId&quot;:&quot;&quot;, &quot;principals&quot;: [], &quot;denyAssignmentId&quot;:&quot;&quot;}]'"

x = txt.split(";")

print(x)


In [ ]:
new = ['p','q','r']
my_str = 'there are two much person a, person b, person c.'
old = ['a','b','c']

for i in range(len(old)):
    my_str = my_str.replace(old[i],new[i])

print(my_str)